In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1336151,2021-06-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1336152,2021-06-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1336153,2021-06-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1336154,2021-06-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
35451,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35453,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35455,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35457,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
35459,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
819939,2021-06-03,Arkansas,Arkansas,5001,2131,37.00,5000,Arkansas,AR,Arkansas,State,3017804
819941,2021-06-04,Arkansas,Arkansas,5001,2136,37.00,5000,Arkansas,AR,Arkansas,State,3017804
819943,2021-06-05,Arkansas,Arkansas,5001,2138,37.00,5000,Arkansas,AR,Arkansas,State,3017804
819945,2021-06-06,Arkansas,Arkansas,5001,2139,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1336151,2021-06-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1336152,2021-06-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1336153,2021-06-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1336154,2021-06-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-06-07') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
503,2021-06-07,Snohomish,Washington,39281,589.00,53061,WA,County,822083,4778.23,71.65
1004,2021-06-07,Cook,Illinois,555060,10934.00,17031,IL,County,5150233,10777.38,212.30
1504,2021-06-07,Orange,California,272447,5083.00,6059,CA,County,3175692,8579.14,160.06
2003,2021-06-07,Maricopa,Arizona,552781,10112.00,4013,AZ,County,4485414,12323.97,225.44
2502,2021-06-07,Los Angeles,California,1245921,24410.00,6037,CA,County,10039107,12410.68,243.15
...,...,...,...,...,...,...,...,...,...,...,...
1335327,2021-06-07,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08
1335565,2021-06-07,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1335772,2021-06-07,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1335975,2021-06-07,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
503,2021-06-07,Snohomish,Washington,39281,589.00,53061,WA,County,822083,4778.23,71.65,71.65,4778.23
1004,2021-06-07,Cook,Illinois,555060,10934.00,17031,IL,County,5150233,10777.38,212.30,212.30,10777.38
1504,2021-06-07,Orange,California,272447,5083.00,6059,CA,County,3175692,8579.14,160.06,160.06,8579.14
2003,2021-06-07,Maricopa,Arizona,552781,10112.00,4013,AZ,County,4485414,12323.97,225.44,225.44,12323.97
2502,2021-06-07,Los Angeles,California,1245921,24410.00,6037,CA,County,10039107,12410.68,243.15,243.15,12410.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335327,2021-06-07,Wheeler,Oregon,34,1.00,41069,OR,County,1332,2552.55,75.08,75.08,2552.55
1335565,2021-06-07,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1335772,2021-06-07,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1335975,2021-06-07,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
857440,2021-06-07,Hale,Alabama,2260,78.00,1065,AL,County,14651,15425.57,532.39,532.39,15425.57,1
933485,2021-06-07,Clarke,Alabama,3527,61.00,1025,AL,County,23622,14931.00,258.23,258.23,14931.00,2
670581,2021-06-07,Lowndes,Alabama,1418,54.00,1085,AL,County,9726,14579.48,555.21,555.21,14579.48,3
594467,2021-06-07,Etowah,Alabama,14125,361.00,1055,AL,County,102268,13811.75,352.99,352.99,13811.75,4
498696,2021-06-07,Franklin,Alabama,4296,83.00,1059,AL,County,31362,13698.11,264.65,264.65,13698.11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264866,2021-06-07,Platte,Wyoming,666,11.00,56031,WY,County,8393,7935.18,131.06,131.06,7935.18,19
818202,2021-06-07,Converse,Wyoming,1018,17.00,56009,WY,County,13822,7365.07,122.99,122.99,7365.07,20
1017727,2021-06-07,Lincoln,Wyoming,1414,12.00,56023,WY,County,19830,7130.61,60.51,60.51,7130.61,21
1133103,2021-06-07,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
670581,2021-06-07,Lowndes,Alabama,1418,54.00,1085,AL,County,9726,14579.48,555.21,555.21,14579.48,3,1
857440,2021-06-07,Hale,Alabama,2260,78.00,1065,AL,County,14651,15425.57,532.39,532.39,15425.57,1,2
592707,2021-06-07,Clay,Alabama,1592,59.00,1027,AL,County,13235,12028.71,445.79,445.79,12028.71,20,3
284852,2021-06-07,Walker,Alabama,7313,279.00,1127,AL,County,63521,11512.73,439.22,439.22,11512.73,28,4
723258,2021-06-07,Greene,Alabama,932,34.00,1063,AL,County,8111,11490.57,419.18,419.18,11490.57,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721505,2021-06-07,Hot Springs,Wyoming,370,3.00,56017,WY,County,4413,8384.32,67.98,67.98,8384.32,17,19
992298,2021-06-07,Uinta,Wyoming,2290,13.00,56041,WY,County,20226,11322.06,64.27,64.27,11322.06,5,20
1017727,2021-06-07,Lincoln,Wyoming,1414,12.00,56023,WY,County,19830,7130.61,60.51,60.51,7130.61,21,21
667947,2021-06-07,Albany,Wyoming,4540,15.00,56001,WY,County,38880,11676.95,38.58,38.58,11676.95,4,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.65,4778.23,20,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.65,4778.23,20,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.65,4778.23,20,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.65,4778.23,20,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,71.65,4778.23,20,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333029,2021-06-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1333030,2021-06-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1333031,2021-06-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1333032,2021-06-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
853884,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15425.57,2,1,1.00
853885,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15425.57,2,1,0.00
853886,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15425.57,2,1,0.00
853887,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15425.57,2,1,0.00
853888,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15425.57,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034811,2021-06-03,Crook,Wyoming,448,12.00,56011,WY,County,7584,5907.17,158.23,158.23,5907.17,6,23,1.00
1034812,2021-06-04,Crook,Wyoming,448,12.00,56011,WY,County,7584,5907.17,158.23,158.23,5907.17,6,23,0.00
1034813,2021-06-05,Crook,Wyoming,448,12.00,56011,WY,County,7584,5907.17,158.23,158.23,5907.17,6,23,0.00
1034814,2021-06-06,Crook,Wyoming,448,12.00,56011,WY,County,7584,5907.17,158.23,158.23,5907.17,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
667895,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14579.48,1,3,1.00,0.00
667896,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14579.48,1,3,0.00,0.00
667897,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14579.48,1,3,0.00,0.00
667898,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14579.48,1,3,0.00,0.00
667899,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14579.48,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281702,2021-06-03,Teton,Wyoming,3785,9.00,56039,WY,County,23464,16131.09,38.36,38.36,16148.14,23,1,0.00,0.00
281703,2021-06-04,Teton,Wyoming,3787,9.00,56039,WY,County,23464,16139.62,38.36,38.36,16148.14,23,1,2.00,0.00
281704,2021-06-05,Teton,Wyoming,3787,9.00,56039,WY,County,23464,16139.62,38.36,38.36,16148.14,23,1,0.00,0.00
281705,2021-06-06,Teton,Wyoming,3787,9.00,56039,WY,County,23464,16139.62,38.36,38.36,16148.14,23,1,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-06-07') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
340124,2021-06-07,Imperial,California,28762,738.00,6025,CA,County,181215,15871.75,407.25,407.25,15871.75,1,2,-2.00,1.00
2502,2021-06-07,Los Angeles,California,1245921,24410.00,6037,CA,County,10039107,12410.68,243.15,243.15,12410.68,2,5,133.00,-1.00
169210,2021-06-07,San Bernardino,California,298697,4773.00,6071,CA,County,2180085,13701.16,218.94,218.94,13701.16,3,4,21.00,1.00
596179,2021-06-07,Inyo,California,1432,38.00,6027,CA,County,18039,7938.36,210.65,210.65,7938.36,4,25,0.00,0.00
75593,2021-06-07,Stanislaus,California,63041,1068.00,6099,CA,County,550660,11448.26,193.95,193.95,11448.26,5,9,60.00,0.00
34992,2021-06-07,Riverside,California,301013,4618.00,6065,CA,County,2470546,12184.07,186.92,186.92,12184.07,6,7,84.00,2.00
60125,2021-06-07,San Joaquin,California,74161,1423.00,6077,CA,County,762148,9730.52,186.71,186.71,9730.52,7,17,47.00,0.00
93750,2021-06-07,Tulare,California,49936,848.00,6107,CA,County,466195,10711.40,181.90,181.90,10711.40,8,10,2.00,1.00
34076,2021-06-07,Fresno,California,102642,1717.00,6019,CA,County,999101,10273.44,171.85,171.85,10273.44,9,14,16.00,0.00
540189,2021-06-07,Merced,California,32215,473.00,6047,CA,County,277680,11601.48,170.34,170.34,11601.48,10,8,22.00,2.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1264417,2021-06-07,Lassen,California,5748,24.00,6035,CA,County,30573,18800.90,78.50,78.50,18800.90,36,1,0.00,0.00
340124,2021-06-07,Imperial,California,28762,738.00,6025,CA,County,181215,15871.75,407.25,407.25,15871.75,1,2,-2.00,1.00
723200,2021-06-07,Kings,California,23124,247.00,6031,CA,County,152940,15119.65,161.50,161.50,15119.65,11,3,2.00,0.00
169210,2021-06-07,San Bernardino,California,298697,4773.00,6071,CA,County,2180085,13701.16,218.94,218.94,13701.16,3,4,21.00,1.00
2502,2021-06-07,Los Angeles,California,1245921,24410.00,6037,CA,County,10039107,12410.68,243.15,243.15,12410.68,2,5,133.00,-1.00
212805,2021-06-07,Kern,California,110395,1398.00,6029,CA,County,900202,12263.36,155.30,155.30,12263.36,14,6,23.00,0.00
34992,2021-06-07,Riverside,California,301013,4618.00,6065,CA,County,2470546,12184.07,186.92,186.92,12184.07,6,7,84.00,2.00
540189,2021-06-07,Merced,California,32215,473.00,6047,CA,County,277680,11601.48,170.34,170.34,11601.48,10,8,22.00,2.00
75593,2021-06-07,Stanislaus,California,63041,1068.00,6099,CA,County,550660,11448.26,193.95,193.95,11448.26,5,9,60.00,0.00
93750,2021-06-07,Tulare,California,49936,848.00,6107,CA,County,466195,10711.40,181.90,181.90,10711.40,8,10,2.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
340124,2021-06-07,Imperial,California,28762,738.00,6025,CA,County,181215,15871.75,407.25,407.25,15871.75,1,2,-2.00,1.00
2502,2021-06-07,Los Angeles,California,1245921,24410.00,6037,CA,County,10039107,12410.68,243.15,243.15,12410.68,2,5,133.00,-1.00
169210,2021-06-07,San Bernardino,California,298697,4773.00,6071,CA,County,2180085,13701.16,218.94,218.94,13701.16,3,4,21.00,1.00
596179,2021-06-07,Inyo,California,1432,38.00,6027,CA,County,18039,7938.36,210.65,210.65,7938.36,4,25,0.00,0.00
75593,2021-06-07,Stanislaus,California,63041,1068.00,6099,CA,County,550660,11448.26,193.95,193.95,11448.26,5,9,60.00,0.00
34992,2021-06-07,Riverside,California,301013,4618.00,6065,CA,County,2470546,12184.07,186.92,186.92,12184.07,6,7,84.00,2.00
60125,2021-06-07,San Joaquin,California,74161,1423.00,6077,CA,County,762148,9730.52,186.71,186.71,9730.52,7,17,47.00,0.00
93750,2021-06-07,Tulare,California,49936,848.00,6107,CA,County,466195,10711.40,181.90,181.90,10711.40,8,10,2.00,1.00
34076,2021-06-07,Fresno,California,102642,1717.00,6019,CA,County,999101,10273.44,171.85,171.85,10273.44,9,14,16.00,0.00
540189,2021-06-07,Merced,California,32215,473.00,6047,CA,County,277680,11601.48,170.34,170.34,11601.48,10,8,22.00,2.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,407.25,15871.75,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4462,06/03/21,Lassen,5748,24.00,18800.90,78.50,78.50,18800.90,36,1,3.00,0.00
4463,06/04/21,Lassen,5748,24.00,18800.90,78.50,78.50,18800.90,36,1,0.00,0.00
4464,06/05/21,Lassen,5748,24.00,18800.90,78.50,78.50,18800.90,36,1,0.00,0.00
4465,06/06/21,Lassen,5748,24.00,18800.90,78.50,78.50,18800.90,36,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,407.25,15871.75,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,407.25,15871.75,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4548,06/03/21,Merced,32183,471.00,11589.96,169.62,170.34,11601.48,10,8,5.00,0.00
4549,06/04/21,Merced,32193,471.00,11593.56,169.62,170.34,11601.48,10,8,10.00,0.00
4550,06/05/21,Merced,32193,471.00,11593.56,169.62,170.34,11601.48,10,8,0.00,0.00
4551,06/06/21,Merced,32193,471.00,11593.56,169.62,170.34,11601.48,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)